# Практика
Используемые библиотеки

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [97]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


In [130]:
# headers={
#     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
# }
# def get_content(url):
#     with requests.Session() as req:
#         req.headers.update(headers)
#         r = req.get(url).content
#     return r

# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = get_content(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
df_load = pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';' , index_col=0)
display(df_load)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [131]:
# Очистка данных
df = df_load.copy(deep=True)
df['limit'] = df['Норматив'].apply(lambda x: x.replace('-', ' '))
df['limit'] = df['limit'].apply(lambda x: x.replace(',', '.'))
df['limit'] = df['limit'].apply(lambda x: x.replace(' ', '_', 1))
df['limit'] = df['limit'].apply(lambda x: x.split(' ')[0])
#df#.head(2)
#df.tail(3)

In [132]:
# Очистка данных
df['total'] = df['Норматив'].apply(lambda x: x.replace(',', '.'))
df['total'] = df['total'].apply(lambda x: x.split(' ')[-1])
df['total'] = df['total'].apply(lambda x: x.split('-'))
df['total'] = df['total'].apply(lambda x: [0] + x if len(x)==1 else x)
df['total'] = df['total'].apply(lambda x: list(map(float, x)))

df['Результат анализа'] = df['Результат анализа'].apply(lambda x: 0 if re.search('[а-яА-Я]', x) !=None else x)
df['Результат анализа'] = df['Результат анализа'].astype(float)
df['total'] = df.apply(lambda row: row['total'] + [row['Результат анализа']] + [row['limit']], axis=1)
#df

In [133]:
df2['total'] = df['total'].apply(lambda x: (True if x[0] <= x[2] <=x[1] else False)
                                                    if x[3]=='в_пределах' else 
                                                    (True if x[2] <=x[1] else False))
df2

,Единица измерений,Результат анализа,Норматив,total
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,True
Запах,баллы,1,не более 2-3,True
Цветность,градусы,б/цвета,не более 30,True
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
Нитраты (по NO3),мг/дм3,24,не более 45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
Хлориды (Cl),мг/дм3,200,не более 350,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [1]:
import math

#Исходные данные
oranges = 5
apples = 4
num_fruits_to_choose = 3

# Из документации math: 
# количество способов выбрать k элементов из n элементов без повторений и без указания порядка(math.comb(n, k)) 

# Количество способов выбрать апельсины
num_ways_to_choose_oranges = math.comb(oranges, num_fruits_to_choose)
# общее количество способов выбора
total_num_ways_to_choose = math.comb(oranges + apples, num_fruits_to_choose)

# вероятность, что все три выбранных фрукта будут апельсинами
probability = num_ways_to_choose_oranges / total_num_ways_to_choose
probability

0.11904761904761904

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [5]:
parts = 10 # количество деталей
nonstandard_parts = 3 # количество Нестандартных деталей
parts_to_check = 2 # количество выборок

In [6]:
standard_parts = parts - nonstandard_parts

In [7]:
# генерация случайных выборок деталей
samples = np.random.choice([0, 1], size=(10000000, parts), 
                                p=[standard_parts/parts, nonstandard_parts/parts])

In [8]:
# подсчет количества выборок, в которых мастер проверил ровно две детали
num_samples_with_two_checks = np.sum(np.sum(samples, axis=1) == parts_to_check)

In [9]:
# вероятность, что мастер проверит ровно две детали
probability = num_samples_with_two_checks / len(samples)
probability

np.float64(0.2333318)